In [1]:
# Library Imports
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [2]:
# Slider class to combine slide and play handler
class Slider:    
    def __init__(self, call_back, descp, default_val=0, minimum=0, maximum=10):
        # Create player and slider
        self.player = widgets.Play(min=minimum, interval=1000, value=default_val)
        self.slider = widgets.FloatSlider(description=descp, min=minimum, max=maximum, step=0.1, continuous_update=True,
                                       layout=widgets.Layout(width='40%'))
        self.link(call_back)
  
    # Link slider and player
    def link(self, on_value_change):
        self.slider.observe(on_value_change, 'value')
        widgets.jslink((self.player, 'value'), (self.slider, 'value'))

In [3]:
# Signal class to generate function points and hold signal parameters
class Signal:    
    def __init__(self, function, color, descp):
        self.function = function
        self.color = color
        self.amplitude = 1
        self.frequency = 1
        self.patch = mpatches.Patch(color=self.color, label=descp)
        
    def update(self, params):
        self.amplitude = params.get("Amplitude", self.amplitude)
        self.frequency = params.get("Frequency", self.frequency)
        
    def generate_points(self, time_values):
        return self.amplitude * self.function(self.frequency * time_values)

In [4]:
# Graph class to hold all plotting variables
class Graph: 
    def __init__(self, slider_list, function_list, title, interval=0.001, limit=6):
        # Create title based on type of plot
        temp = r' of $e^{i \Omega t}, \Omega = $' if interval < 0.5 else r' of $e^{i \omega n}, \omega = $'
        self.title = title + temp
        self.sliders = [Slider(self.call_back, *slide) for slide in slider_list]
        self.functions = [Signal(*funct) for funct in function_list]
        
        self.time_values = np.arange(0, limit, interval)
        self.out = widgets.Output()
        
        self.fig, self.ax = plt.subplots(figsize=(8, 6))
        self.ax.grid(True)
        self.ax.set_ylabel("Amplitude")
        self.ax.axis([0, limit, -10, 10])
        self.ax.axhline(0, linewidth=1, color='black')
        self.ax.axvline(0, linewidth=1, color='black')
        self.ax.legend(handles=[funct.patch for funct in self.functions])
        self.initial_plot(interval)
        plt.close(self.fig)
        self.refresh()
        
    # Create the first plot
    def initial_plot(self, interval):
        params = {} if interval < 0.01 else {'marker':'.', 'linestyle':'None'}
        self.curve = {}
        # Plot the functions
        for funct in self.functions:
            self.curve[funct.color], = self.ax.plot(
                self.time_values, funct.generate_points(self.time_values), color=funct.color, **params)
        # Get all elements to display
        display_list = [widgets.HBox([slide.slider, slide.player]) for slide in self.sliders]
        display_list.append(self.out)
        self.full_output = widgets.VBox(display_list)
        
    # Respond to slider changes
    def call_back(self, change):
        for funct in self.functions:
            funct.update({change["owner"].description : change["new"]})
            if change["owner"].description == "Frequency":
                if len(self.time_values) < 50:
                    self.ax.set_title(self.title + " %.2f radians/sample" % (change["new"]))
                else:
                    self.ax.set_title(self.title + " %.2f radians/second" % (change["new"]))
            self.curve[funct.color].set_ydata(funct.generate_points(self.time_values))
        self.refresh()
    
    # Refresh the plot
    def refresh(self):
        self.out.clear_output(wait=True)
        with self.out:
            display(self.fig)
    
    # Display the plot
    def display_graph(self):
        display(self.full_output)
        self.refresh()
        

In [5]:
# Place all graphs inside tabs
def display_tab(plots, titles):
    tab = widgets.Tab()
    tab.children = plots
    for i in range(len(titles)):
        tab.set_title(i, titles[i])
    display(tab)

In [6]:
# Continuous Time Complex Exponential
cont_cplx = Graph([["Amplitude", 1], ["Frequency", 1]], [[np.cos, "red", "Real"], [np.sin, "blue", "Imaginary"]],
                 "Continuous Time Complex Exponential\nBoth components")
cont_real = Graph([["Amplitude", 1], ["Frequency", 1]], [[np.cos, "red", "Real"]],
                 "Continuous Time Complex Exponential\nReal component")
cont_imag = Graph([["Amplitude", 1], ["Frequency", 1]], [[np.sin, "blue", "Imaginary"]],
                 "Continuous Time Complex Exponential\nImaginary component")
tab_contents = ["Real", "Imaginary", "Both"]

display_tab([cont_real.full_output, cont_imag.full_output, cont_cplx.full_output], tab_contents)

In [7]:
# Discrete Time Complex Exponential
dcrt_cplx = Graph([["Amplitude", 1], ["Frequency", 1]], [[np.cos, "red", "Real"], [np.sin, "blue", "Imaginary"]],
                  "Discrete Time Complex Exponential\nBoth components", 1, 30)
dcrt_real = Graph([["Amplitude", 1], ["Frequency", 1]], [[np.cos, "red", "Real"]],
                  "Discrete Time Complex Exponential\nReal component", 1, 30)
dcrt_imag = Graph([["Amplitude", 1], ["Frequency", 1]], [[np.sin, "blue", "Imaginary"]],
                  "Discrete Time Complex Exponential\nImaginary component" ,1, 30)
tab_contents = ["Real", "Imaginary", "Both"]

display_tab([dcrt_real.full_output, dcrt_imag.full_output, dcrt_cplx.full_output], tab_contents)